# Replication of Aiyagari (1994)

##### Caveat: not self-documented perfectly

#### Youngdoo Choi (yoc005@ucsd.edu), February 2023

In [1]:
import numpy as np
import pandas as pd
from numba import njit

from interpolation import interp
from quantecon.markov.approximation import tauchen

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## 0. Parameterization

In [2]:
# Household parameters
β = 0.96 # discount factor
μ_vec = np.array((1, 3, 5)) # RRA coefficient

# Firm parameters
α = 0.36 # capital share
δ = 0.08 # depreciation rate

# Labor productivity process
lgridsize = 7
ρ_vec = np.array((0, 0.3, 0.6, 0.9)) # serial correlation
σ_vec = np.array((0.2, 0.4)) # variance

# Asset grids
a_min = 0.0; a_max = 200.0; agridsize = 201
curv = 1.5 # curvature
agrid = np.array([a_min + a_max * (a/(agridsize-1))**curv for a in range(agridsize)])

# Etc
max_iter = 10000 # max of iteration
tol = 1e-6 # tolerance

## 1. HH Problem

In [3]:
@njit
def EGM(c_0, r, w, 
        lgrid, lgridsize, agrid, agridsize, Π, β, μ, max_iter, tol):
    # Function for solving HH policy functions by the endogeneous grid method for given prices
    
    # Initial guess
    c_old = c_0
    
    for iter in range(max_iter):
        c_new = np.empty((lgridsize, agridsize))
        
        # Operator
        for (lidx, l) in enumerate(lgrid):
            c_tilde = np.empty(agridsize)
            a_tilde = np.empty(agridsize)
            
            for (aidx, ap) in enumerate(agrid):
                c_tilde[aidx] = (β * (1+r) * np.dot(Π[lidx, :], c_old[:, aidx]**(-μ))) ** (-1/μ)
                a_tilde[aidx] = (c_tilde[aidx] + ap - w*l)/(1+r)
            
            # Check borrowing constraint
            # binding case
            thres_idx = np.where(agrid > a_tilde[0])[0][0]
            c_new[lidx, :thres_idx] = (1+r)*agrid[:thres_idx] + w*l - agrid[0] 
            # non-binding case
            c_tilde_interp = lambda a: interp(a_tilde, c_tilde, a)
            c_new[lidx, thres_idx:] = c_tilde_interp(agrid[thres_idx:])
            
        # Updating
        if np.max(np.abs(c_new - c_old)) < tol:
            c_star = c_new        
            ap_star = np.empty((lgridsize, agridsize))
            for i in range(lgridsize):
                ap_star[i, :] = (1+r)*agrid + w*lgrid[i] - c_star[i, :]
            return c_star, ap_star
        else:            
            c_old = c_new
            
    # Check convergence
    if iter == max_iter-1:
        print("Error: No convergence in EGM function")

In [5]:
%%time 
# Example: EGM

# Give necessaries
σ = 0.2
ρ = 0.6
μ = 3
r = 0.04
c_0 = np.ones((lgridsize, agridsize))

# Wage
w = (1-α)*((r+δ)/α)**(α/(α-1))

# Labor productivity grids
MC = tauchen(lgridsize, ρ, σ*(1-ρ**2)**(1/2), mu=0, n_std=3)
Π = MC.P
lgrid_temp = np.exp(MC.state_values); Π_stat = MC.stationary_distributions[0]
lgrid = lgrid_temp / np.sum(lgrid_temp * Π_stat)

# Solve
c_star, ap_star = EGM(c_0, r, w, lgrid, lgridsize, agrid, agridsize, Π, β, μ, max_iter, tol)

CPU times: user 713 ms, sys: 1.67 ms, total: 715 ms
Wall time: 726 ms


## 2. Stationary Distributions

In [6]:
@njit
def stat_dist(mu_0, ap_star, 
              lgridsize, agrid, agridsize, Π, max_iter, tol):
    # Function for solving the stationary distribution for given prices and policy functions
    
    # Initial guess
    mu_old = mu_0
    
    for iter in range(max_iter):
        mu_new = np.zeros((lgridsize, agridsize))
        
        # Operator        
        for lidx in range(lgridsize):
            for aidx in range(agridsize):
                
                if ap_star[lidx, aidx] > agrid[-1]:
                    mu_new[:, -1] += mu_old[lidx, aidx] * Π[lidx, :]
                else:
                    idx_r = np.where(agrid > ap_star[lidx, aidx])[0][0]                    
                    idx_l = idx_r - 1
                    weight = (agrid[idx_r]-ap_star[lidx, aidx]) / (agrid[idx_r]-agrid[idx_l])
                    mu_new[:, idx_l] += mu_old[lidx, aidx] * Π[lidx, :] * weight
                    mu_new[:, idx_r] += mu_old[lidx, aidx] * Π[lidx, :] * (1-weight)
                  
        # Updating
        if np.max(np.abs(mu_new - mu_old)) < tol:
            mu_star = mu_new
            return mu_star
        else:
            mu_old = mu_new            
    
    # Check convergence
    if iter == max_iter-1:
        print("No convergence in stat_dist function")

In [8]:
%%time
# Example: stat_dist

# Give necessaries
mu_0 = np.zeros((lgridsize, agridsize)); mu_0[int((lgridsize+1)/2), 0] = 1

# Solve
mu_star = stat_dist(mu_0, ap_star, lgridsize, agrid, agridsize, Π, max_iter, tol)

CPU times: user 1.06 s, sys: 2.56 ms, total: 1.06 s
Wall time: 1.08 s


## 3. Equilibrium Interest Rates

In [9]:
@njit
def Eqm_r(r_0, 
          μ, lgrid, lgridsize, agrid, agridsize, Π, β, α, δ, max_iter, tol, 
          tol_eqm=1e-3, procedure=0):
    # Function for solving equilibrium interest rates
        
    # Initial guess
    r_old = r_0
    c_0 = np.ones((lgridsize, agridsize))
    mu_0 = np.zeros((lgridsize, agridsize)); mu_0[int((lgridsize+1)/2), 0] = 1
    r_high = (1-β)/β - 0.0002
    r_low = -δ
    
    for iter in range(max_iter):
        
        # Operator
        w = (1-α)*((r_old+δ)/α)**(α/(α-1))
        c_star, ap_star = EGM(c_0, r_old, w, lgrid, lgridsize, agrid, agridsize, Π, β, μ, max_iter, tol)
        mu_star = stat_dist(mu_0, ap_star, lgridsize, agrid, agridsize, Π, max_iter, tol)
        
        # Asset market
        K_supply = np.sum(np.sum(mu_star, 0)*agrid)
        K_demand = ((r_old+δ)/α)**(1/(α-1))
            
        if procedure:
            print([r_old*100, K_supply-K_demand, K_supply, K_demand])
            
        # Updating
        if np.abs(K_supply-K_demand) < tol_eqm:
            r_eqm = r_old
            s_rate = (δ*α)/(r_eqm+δ)*100
            return r_eqm, s_rate
        elif K_supply > K_demand:
            r_high = r_old
            r_old = (r_old+r_low)/2
        elif K_demand > K_supply:
            r_low = r_old
            r_old = (r_old+r_high)/2
    
    # Check convergence
    if iter == max_iter-1:
        print("No convergence in Eqm_r function")

In [11]:
%%time
# Example: Eqm_r

# Give parameters
σ = 0.2
ρ = 0.6
μ = 3
r_0 = 0.04

# Labor productivity grids
MC = tauchen(lgridsize, ρ, σ*(1-ρ**2)**(1/2), mu=0, n_std=3)
Π = MC.P
lgrid_temp = np.exp(MC.state_values); Π_stat = MC.stationary_distributions[0]
lgrid = lgrid_temp / np.sum(lgrid_temp * Π_stat)

# Solve
r_eqm, s_rate = Eqm_r(r_0, μ, lgrid, lgridsize, agrid, agridsize, Π, β, α, δ, max_iter, tol, tol_eqm=1e-3, procedure=1)
r_eqm*100, s_rate

[4.0, 3.9481294822805353, 9.513600948551536, 5.565471466271001]
[-2.0, -15.800767384079489, 0.6377001676590234, 16.438467551738512]
[1.0, -7.738934743601368, 0.9851271177206937, 8.724061861322062]
[2.5, -5.303759019867439, 1.552928813519477, 6.856687833386916]
[3.25, -3.763200420027859, 2.3927744902895105, 6.1559749103173695]
[3.6250000000000004, -2.278870453541703, 3.569651911468178, 5.848522365009881]
[3.8125000000000004, -0.6675123115679513, 5.036606479431245, 5.704118790999196]
[3.90625, 0.8912852022290769, 6.5253809606060384, 5.6340957583769615]
[3.8593750000000004, 0.003597712590221569, 5.67252768139592, 5.668929968805698]
[3.8359375, -0.35370142534288984, 5.332778315303208, 5.686479740646098]
[3.8476562500000004, -0.1810813352417986, 5.496612398988798, 5.677693734230597]
[3.853515625000001, -0.09044045985217952, 5.5828686164375965, 5.673309076289776]
[3.856445312500001, -0.043798599592612675, 5.627320229758754, 5.671118829351367]
[3.857910156250001, -0.019961691231376477, 5.6500

(3.85919189453125, 24.284959933299366)

## 4. Replicate Table II

In [13]:
%%time

# Table II_A
σ = σ_vec[0]
Table_IIA = pd.DataFrame(index = ρ_vec, columns = μ_vec)
r_0 = 0.03

for (ρ_idx, ρ) in enumerate(ρ_vec):
    for (μ_idx, μ) in enumerate(μ_vec):
        
        # Labor productivity grids
        MC = tauchen(lgridsize, ρ, σ*(1-ρ**2)**(1/2), mu=0, n_std=3)
        Π = MC.P
        lgrid_temp = np.exp(MC.state_values); Π_stat = MC.stationary_distributions[0]
        lgrid = lgrid_temp / np.sum(lgrid_temp * Π_stat)

        # Solve
        r_eqm, s_rate = Eqm_r(r_0, μ, lgrid, lgridsize, agrid, agridsize, Π, β, α, δ, max_iter, tol, tol_eqm=1e-3, procedure=0)
        Table_IIA.iloc[ρ_idx, μ_idx] = (round(r_eqm*100, 4), round(s_rate, 4))
        print((r_eqm*100, s_rate))
        
Table_IIA

(4.139807942708337, 23.72360430734693)
(4.071640625000003, 23.857569070069953)
(3.9859065755208367, 24.02821999198548)
(4.121313883463546, 23.75980052730941)
(4.004873046875003, 23.990257862407756)
(3.85902018229167, 24.285311566469236)
(4.080948893229171, 23.83918701629564)
(3.859160156250003, 24.285024926340885)
(3.586210937500003, 24.857134187662457)
(3.947133789062503, 24.106200289115492)
(3.3555338541666675, 25.362083694050597)
(2.653564453125, 27.033205765749248)
CPU times: user 3min 20s, sys: 356 ms, total: 3min 20s
Wall time: 3min 26s


,1,3,5
0.0,"(4.1398, 23.7236)","(4.0716, 23.8576)","(3.9859, 24.0282)"
0.3,"(4.1213, 23.7598)","(4.0049, 23.9903)","(3.859, 24.2853)"
0.6,"(4.0809, 23.8392)","(3.8592, 24.285)","(3.5862, 24.8571)"
0.9,"(3.9471, 24.1062)","(3.3555, 25.3621)","(2.6536, 27.0332)"


In [15]:
%%time

# Table II_B
σ = σ_vec[1]
Table_IIB = pd.DataFrame(index = ρ_vec, columns = μ_vec)
r_0 = 0.03

for (ρ_idx, ρ) in enumerate(ρ_vec):
    for (μ_idx, μ) in enumerate(μ_vec):
        
        # Labor productivity grids
        MC = tauchen(lgridsize, ρ, σ*(1-ρ**2)**(1/2), mu=0, n_std=3)
        Π = MC.P
        lgrid_temp = np.exp(MC.state_values); Π_stat = MC.stationary_distributions[0]
        lgrid = lgrid_temp / np.sum(lgrid_temp * Π_stat)

        # Solve
        r_eqm, s_rate = Eqm_r(r_0, μ, lgrid, lgridsize, agrid, agridsize, Π, β, α, δ, max_iter, tol, tol_eqm=1e-3, procedure=0)
        Table_IIB.iloc[ρ_idx, μ_idx] = (round(r_eqm*100, 4), round(s_rate, 4))
        print((r_eqm*100, s_rate))

Table_IIB

(4.05134440104167, 23.897748700560445)
(3.7603385416666706, 24.489090937273712)
(3.4140429687500014, 25.232076030246454)
(3.967569986979169, 24.06503578532206)
(3.4697526041666684, 25.10952153365339)
(2.9033203124999996, 26.413972234661887)
(3.79561197916667, 24.4158590930817)
(2.8939208984375, 26.43676254720258)
(1.9680786132812502, 28.892227998310034)
(3.3885677083333343, 25.2885180450973)
(1.498443603515625, 30.32075695995117)
(-0.10247802734375012, 36.467135007303334)
CPU times: user 1min 3s, sys: 92 ms, total: 1min 3s
Wall time: 1min 4s


,1,3,5
0.0,"(4.0513, 23.8977)","(3.7603, 24.4891)","(3.414, 25.2321)"
0.3,"(3.9676, 24.065)","(3.4698, 25.1095)","(2.9033, 26.414)"
0.6,"(3.7956, 24.4159)","(2.8939, 26.4368)","(1.9681, 28.8922)"
0.9,"(3.3886, 25.2885)","(1.4984, 30.3208)","(-0.1025, 36.4671)"
